In [1]:
%matplotlib inline

# ニューラルネットワークの構築

ニューラルネットワークは、データに対して演算を行う層やモジュールで構成されています。
[torch.nn](https://pytorch.org/docs/stable/nn.html)名前空間は、独自のニューラルネットワークを構築するために必要なすべてのビルディングブロックを提供します。
PyTorchの全てのモジュールは、[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)をサブクラス化しています。
ニューラルネットワークは、モジュール自体が他のモジュール(層)で構成されています。この入れ子構造により
複雑なアーキテクチャを簡単に構築・管理することができます。

以下のセクションでは、FashionMNISTデータセットの画像を分類するためのニューラルネットワークを構築します。


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## トレーニング用のハードウェアデバイスの取得

GPUのようなハードウェア・アクセラレータでモデルを学習できるようにしたいと思います。
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html)が利用可能かどうかチェックしてみましょう。
利用可能でなければCPUを使い続けることになります。

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


## クラスの定義

ニューラルネットワークを定義するには、`nn.Module` をサブクラス化します。`__init__`でニューラルネットワークのレイヤーを初期化します。各 `nn.Module` サブクラスは入力データに対する操作を `forward` メソッドで実装しています。

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

`NeuralNetwork`のインスタンスを作成し、それを`device`に移動させて、その構造を表示します。

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


モデルを使用するには、入力データを渡します。これにより、モデルの`forward`が実行されます。
それらに加えていくつかの[バックグラウンド操作](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866)が実行されます。
`model.forward()`を直接呼ばないでください!

入力に対してモデルを呼び出すと、各クラスの生の予測値を持つ10次元のテンソルが返されます。
これを `nn.Softmax` モジュールのインスタンスに渡すことで，予測密度を得ることができます．


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4])


## モデルのレイヤー

FashionMNISTモデルのレイヤーを分解してみましょう。説明するために以下を行います。
サイズ**28x28**の3つの画像のサンプルミニバッチを取り、それをネットワークに通すとどうなるかを見てみましょう。


In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)でレイヤーを初期化し、各2D 28x28画像を784ピクセル値の連続した配列に変換します（ミニバッチの次元（dim=0）は維持されます）。



In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

[linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)は保存されている重みとバイアスを使って、入力に線形変換を施すモジュールです。


In [9]:
layer1 = nn.Linear(in_features=28 * 28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

非線形活性化は、モデルの入力と出力の間の複雑なマッピングを作成するものです。
非線形活性化は、線形変換の後に適用され、非線形性を導入し、ニューラルネットワークが様々な現象を学習するのに役立ちます。

このモデルでは、線形層の間に[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)を使用しています。
しかし、モデルに非線形性を導入するための他のアクティベーションもあります。

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0701, -0.3081,  0.3626, -0.0073,  0.3439, -0.1468,  0.2568, -0.5668,
         -0.3017,  0.5958, -0.2099,  0.2591, -0.0570,  0.1017,  0.1859, -0.5327,
          0.0203, -0.4696, -0.4637,  0.0412],
        [ 0.1021, -0.0878,  0.4313, -0.1253,  0.3443,  0.1254,  0.1518, -0.1452,
         -0.1431,  0.5728,  0.0506,  0.3015, -0.0988,  0.1512, -0.0285, -0.1421,
         -0.1407, -0.3318, -0.3642,  0.1773],
        [-0.3462, -0.4096,  0.5588,  0.0210,  0.4451, -0.0928,  0.4437, -0.1128,
         -0.3589,  0.4629, -0.1584,  0.2374,  0.0369,  0.1840,  0.0866, -0.6190,
          0.0120, -0.3414, -0.7449,  0.2643]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.3626, 0.0000, 0.3439, 0.0000, 0.2568, 0.0000, 0.0000,
         0.5958, 0.0000, 0.2591, 0.0000, 0.1017, 0.1859, 0.0000, 0.0203, 0.0000,
         0.0000, 0.0412],
        [0.1021, 0.0000, 0.4313, 0.0000, 0.3443, 0.1254, 0.1518, 0.0000, 0.0000,
         0.5728, 0.0506, 0.3015, 0.0000, 0.1512, 0.000

### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)は、順序付けられたモジュールのコンテナです。データは定義されたとおりの順序ですべてのモジュールに渡されます。シーケンシャルコンテナを使って、`seq_modules`のような素早いネットワークを組むことができます。

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

ニューラルネットワークの最後の線形層は、`logits`（`-infty`, `infty` の生の値）を返して
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)モジュールに渡されます。logitsは、各クラスに対するモデルの予測密度を表す値\[0, 1\]にスケーリングされます。パラメータ `dim` は，値の合計が1になるような次元を示します。

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## モデルパラメータ

ニューラルネットワークの多くの層は，*パラメータ化*されています．すなわち，学習時に最適化された重み，バイアスが関連付けられており、学習時に最適化されます。`nn.Module` をサブクラス化することでモデルオブジェクト内で定義されたすべてのフィールドを自動的に追跡し、モデルの `parameters()` や `named_parameters()` メソッドを使って、すべてのパラメータにアクセスできるようになります。

この例では，各パラメータを繰り返し処理し，そのサイズと値のプレビューを表示しています。

In [14]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 7.2443e-03, -2.1236e-02,  1.7604e-02,  ..., -1.0633e-02,
         -1.5831e-02, -1.8122e-03],
        [ 3.4512e-02, -2.5456e-02,  3.4360e-02,  ...,  6.9336e-03,
          1.3214e-05,  6.4570e-03]], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0254,  0.0086], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0265, -0.0267, -0.0338,  ..., -0.0372,  0.0042, -0.0288],
        [-0.0150, -0.0190, -0.0026,  ..., -0.0379,  0.0133,  0.02

# 知識の確認

1. PyTorchの全てのニューラルネットワークモジュールの基本クラスはtorch.nn.Moduleです。

* 真
  > 正解!

* 偽
  > 間違っています。PyTorchの全てのニューラルネットワークモジュールの基本クラスはtorch.nn.Moduleです。
